# Assignment W02 - B17

Import data from file

In [43]:
# Importing the file
file = open("W02_data_groceries.csv")
s = file.read()
file.close()

Get all baskets in the list baskets

In [44]:
# Seperating each transaction
baskets = s.split("\n")
# Each transaction is a list that contains all the items
baskets = [x.split(",") for x in baskets]

Get the list of all items to be placed in shelves (some are already there, others not yet)

In [45]:
# SKUs is a sorted list of all the items including the new products
SKUs = s.replace("\n",",")
SKUs = SKUs.split(",")
SKUs = list(set(SKUs))
SKUs.remove("")
SKUs.sort()

In [46]:
print('Total number of products:', len(SKUs))

Total number of products: 169


In [47]:
import math 
print('There are 7 shelves, so each one should have more o less', math.floor(169/7), 'items')

There are 7 shelves, so each one should have more o less 24 items


Items that are already in the shelves (there is no repeated item)

In [48]:
shelf1 = ['shopping bags','newspapers','long life bakery product','cling film/bags','flower (seeds)','pet care','photo/film','candles','dog food','rice','instant coffee','kitchen towels','light bulbs','preservation products','sound storage medium']
shelf2 = ['soda','bottled water','bottled beer','canned beer','fruit/vegetable juice','red/blush wine','white wine','liquor','sparkling wine','rum','brandy','cocoa drinks','prosecco','liqueur','whisky']
shelf3 = ['rolls/buns','sausage','pastry','brown bread','frankfurter','pork','beef','white bread','waffles','hamburger meat','zwieback','cereals','syrup','fish','honey']
shelf4 = ['napkins','hygiene articles','softener','cleaner','male cosmetics','abrasive cleaner','skin care','bathroom cleaner','decalcifier','hair spray','make up remover','toilet cleaner','baby cosmetics','kitchen utensil','baby food']
shelf5 = ['chocolate','salty snack','specialty chocolate','candy','specialty bar','chewing gum','cake bar','chocolate marshmallow','popcorn','nuts/prunes','artif. sweetener','snack products','cookware','cooking chocolate','pudding powder']
shelf6 = ['other vegetables','root vegetables','tropical fruit','citrus fruit','pip fruit','sauces','spices','ketchup','tea','canned fruit','potato products','organic sausage','ready soups','specialty vegetables','salad dressing']
shelf7 = ['whole milk','yogurt','butter','curd','frozen meals','spread cheese','frozen dessert','condensed milk','specialty cheese','frozen potato products','finished products','curd cheese','cream','frozen fruits','frozen chicken']
shelves = [shelf1, shelf2, shelf3, shelf4, shelf5, shelf6, shelf7]

In [60]:
for i, shelf in enumerate(shelves):
    print(i)
    for p in shelf: 
        print(p)

0
shopping bags
newspapers
long life bakery product
cling film/bags
flower (seeds)
pet care
photo/film
candles
dog food
rice
instant coffee
kitchen towels
light bulbs
preservation products
sound storage medium
1
soda
bottled water
bottled beer
canned beer
fruit/vegetable juice
red/blush wine
white wine
liquor
sparkling wine
rum
brandy
cocoa drinks
prosecco
liqueur
whisky
2
rolls/buns
sausage
pastry
brown bread
frankfurter
pork
beef
white bread
waffles
hamburger meat
zwieback
cereals
syrup
fish
honey
3
napkins
hygiene articles
softener
cleaner
male cosmetics
abrasive cleaner
skin care
bathroom cleaner
decalcifier
hair spray
make up remover
toilet cleaner
baby cosmetics
kitchen utensil
baby food
4
chocolate
salty snack
specialty chocolate
candy
specialty bar
chewing gum
cake bar
chocolate marshmallow
popcorn
nuts/prunes
artif. sweetener
snack products
cookware
cooking chocolate
pudding powder
5
other vegetables
root vegetables
tropical fruit
citrus fruit
pip fruit
sauces
spices
ketchup
t

In [49]:
import itertools

already_placed_items = list(itertools.chain.from_iterable(shelves))

In [50]:
print('Total number of items in shelves: ', len(already_placed_items))

Total number of items in shelves:  105


Calculate items to be placed

In [51]:
items_to_be_placed = [ item for item in SKUs if item not in already_placed_items] 

In [52]:
print('Total number of itemsto be placed in shelves: ', len(items_to_be_placed))

Total number of itemsto be placed in shelves:  64


## Strategy #1 to place all remaining items in shelves 

For each item to be placed, we want to know what is the best shelf in order to improve profits. That is why we created a function that computes a score for each possible location. Then we applied it and placed each item in the shelf with the maximun score.    
 

Our score function is defined considering the following association rules: support, confidence, lift and leverage. These rules calculate if the items should be placed closely together. So in the score function we applied to each possible pair made by an item to be place and an already placed item.

In [53]:
def get_support_for(item):
    item_count = 0
    for x in baskets:
        if item in x:
            item_count += 1
    #print("Support for {0:}: {1:.2f}".format(item,item_count/len(baskets)))
    return item_count/len(baskets)

In [54]:
def get_association_rules_for_pair(pair_of_items):
    item1 = pair_of_items[0]
    item2 = pair_of_items[1]
    item1_count = 0
    item2_count = 0
    item_combi_count = 0
    count_baskets = len(baskets)
    for x in baskets:
        if item1 in x:
            item1_count += 1
            if item2 in x:
                item_combi_count += 1
        if item2 in x:
            item2_count += 1
    support = item_combi_count/count_baskets
    lift = item_combi_count / item1_count/ item2_count * count_baskets
    leverage = item_combi_count / count_baskets - item1_count * item2_count / count_baskets**2
    confidence = item_combi_count / item1_count
    #print("KPIs for combination " + item1 + "/" + item2 + ":")
    #print("Support: {0:.3f}".format(support))
    #print("Lift: {0:.3f}".format(lift))
    #print("Leverage: {0:.3f}".format(leverage))
    return support, lift, leverage, confidence

In [55]:
def make_pair_of_with_all_from(unplaced_item, shelf_items):
    return {(unplaced_item,already_placed_item) for already_placed_item in shelf_items}

In [56]:
import numpy as np 

def score_item_placed_in_shelf(pairs_used_to_calculate_score):
    scores = np.zeros(len(pairs_used_to_calculate_score))
    for pair_number, pair in enumerate(pairs_used_to_calculate_score):
        support, lift, leverage, confidence = get_association_rules_for_pair(pair)
        pair_closeness= 0 
        if lift>1 and leverage>0:
            pair_closeness += confidence + support 
        if lift<1 and leverage <0:
            pair_closeness -= confidence + support
        scores[pair_number] = pair_closeness
    return scores.mean()

In [57]:
shelves_for_items = {}
scores = [0]*len(items_to_be_placed)
for item_to_be_placed in items_to_be_placed:
    for shelf_number, shelf in enumerate(shelves):
        shelf_pairs_to_compute_score = make_pair_of_with_all_from(item_to_be_placed, shelf)
        scores[shelf_number] = score_item_placed_in_shelf(shelf_pairs_to_compute_score)
    shelves_for_items[item_to_be_placed] = scores.index(max(scores))
    

In [58]:
for shelf in set(shelves_for_items.values()):
    print(shelf)
    for product in shelves_for_items.keys():
        if shelves_for_items[product] == shelf:
            print(product)

0
flower soil/fertilizer
1
liquor (appetizer)
misc. beverages
2
Instant food products
bags
berries
beverages
canned fish
cream cheese
dental care
dish cleaner
domestic eggs
ham
jam
liver loaf
margarine
mayonnaise
meat
mustard
nut snack
organic products
packaged fruit/vegetables
pickled vegetables
processed cheese
seasonal products
semi-finished bread
sliced cheese
soap
soft cheese
sugar
tidbits
5
UHT-milk
baking powder
butter milk
canned vegetables
chicken
detergent
dishes
female sanitary products
flour
frozen fish
frozen vegetables
grapes
herbs
ice cream
oil
onions
roll products
rubbing alcohol
salt
soups
specialty fat
sweet spreads
turkey
vinegar
whipped/sour cream
6
cat food
coffee
dessert
hard cheese
house keeping products
meat spreads
pasta
potted plants


Let's count how many items should we put in each shelf according with Strategy #1

In [17]:
from collections import Counter

res = Counter(shelves_for_items.values())
for shelf_number in range(len(shelves)):
    print('In shelf #', shelf_number, ', we should add', res[shelf_number], ' items') 


In shelf # 0 , we should add 1  items
In shelf # 1 , we should add 2  items
In shelf # 2 , we should add 28  items
In shelf # 3 , we should add 0  items
In shelf # 4 , we should add 0  items
In shelf # 5 , we should add 25  items
In shelf # 6 , we should add 8  items


## Strategy #2

Because in strategy #1 we should place most of all new items in shelf #3 and #6 and we should have 24 more o less items per shelf. There is enough evidence to consider a another kind of distribution.

After analizing all items already placed in the shelves, we see that we were able to define a category for each shelf's products.

shelf #1: miscellaneous
shelf #2: beverages
shelf #3: brunch
shelf #4: hygiene
shelf #5: pastries
shelf #6: vegetables
shelf #7: daires

First, we decided to categorize each new item according to our clasification:


In [18]:
shelf_1_miscellaneous = [items_to_be_placed[0], items_to_be_placed[2], items_to_be_placed[9], items_to_be_placed[21], items_to_be_placed[28], items_to_be_placed[46], items_to_be_placed[48], items_to_be_placed[51]]
shelf_2_beverages = [items_to_be_placed[5], items_to_be_placed[31], items_to_be_placed[37]]
shelf_3_brunch = [items_to_be_placed[7], items_to_be_placed[10], items_to_be_placed[18], items_to_be_placed[22], items_to_be_placed[25], items_to_be_placed[32], items_to_be_placed[33], items_to_be_placed[35], items_to_be_placed[36], items_to_be_placed[61]] 
shelf_4_hygiene = [items_to_be_placed[13], items_to_be_placed[15], items_to_be_placed[16], items_to_be_placed[17], items_to_be_placed[19], items_to_be_placed[28], items_to_be_placed[49], items_to_be_placed[54]]
shelf_5_pastries = [items_to_be_placed[3], items_to_be_placed[14], items_to_be_placed[18], items_to_be_placed[20], items_to_be_placed[30], items_to_be_placed[39], items_to_be_placed[44], items_to_be_placed[52], items_to_be_placed[57], items_to_be_placed[58], items_to_be_placed[59], items_to_be_placed[60]]
shelf_6_vegetables = [items_to_be_placed[4], items_to_be_placed[8], items_to_be_placed[11], items_to_be_placed[23], items_to_be_placed[24], items_to_be_placed[27], items_to_be_placed[34], items_to_be_placed[38], items_to_be_placed[40], items_to_be_placed[41], items_to_be_placed[42], items_to_be_placed[43], items_to_be_placed[45], items_to_be_placed[50], items_to_be_placed[56], items_to_be_placed[62]]
shelf_7_dairies = [items_to_be_placed[1], items_to_be_placed[6], items_to_be_placed[12], items_to_be_placed[14], items_to_be_placed[18], items_to_be_placed[26], items_to_be_placed[29],items_to_be_placed[47], items_to_be_placed[53], items_to_be_placed[55], items_to_be_placed[63]]

shelves_suggestion = [shelf_1_miscellaneous, shelf_2_beverages, shelf_3_brunch, shelf_4_hygiene, shelf_5_pastries, shelf_6_vegetables, shelf_7_dairies]


In [61]:
for i, shelf in enumerate(shelves_suggestion):
    print(i)
    for p in shelf: 
        print(p)

0
Instant food products
bags
cat food
flower soil/fertilizer
house keeping products
potted plants
roll products
seasonal products
1
beverages
liquor (appetizer)
misc. beverages
2
canned fish
chicken
domestic eggs
frozen fish
ham
liver loaf
margarine
meat
meat spreads
turkey
3
dental care
detergent
dish cleaner
dishes
female sanitary products
house keeping products
rubbing alcohol
soap
4
baking powder
dessert
domestic eggs
flour
jam
nut snack
pasta
semi-finished bread
specialty fat
sugar
sweet spreads
tidbits
5
berries
canned vegetables
coffee
frozen vegetables
grapes
herbs
mayonnaise
mustard
oil
onions
organic products
packaged fruit/vegetables
pickled vegetables
salt
soups
vinegar
6
UHT-milk
butter milk
cream cheese
dessert
domestic eggs
hard cheese
ice cream
processed cheese
sliced cheese
soft cheese
whipped/sour cream


In [19]:
count_shelves_suggestions = [len(x) for x in shelves_suggestion]
count_shelves_suggestions

[8, 3, 10, 8, 12, 16, 11]

Second, we consider each item's category to get the score assigned for each item in a particular shelf. Items in their correct category receives a better score.

In [97]:
shelves_for_items = {}
scores = [0]*len(items_to_be_placed)
for item_to_be_placed in items_to_be_placed:
    for shelf_number, shelf in enumerate(shelves):
        shelf_pairs_to_compute_score = make_pair_of_with_all_from(item_to_be_placed, shelf)
        score = score_item_placed_in_shelf(shelf_pairs_to_compute_score)
        if item_to_be_placed in shelves_suggestion[shelf_number]:
            score+=0.052
        scores[shelf_number] = score
    shelves_for_items[item_to_be_placed] = scores.index(max(scores))
    

In [98]:
for shelf in set(shelves_for_items.values()):
    print(shelf)
    for product in shelves_for_items.keys():
        if shelves_for_items[product] == shelf:
            print(product)

0
bags
cat food
flower soil/fertilizer
house keeping products
potted plants
1
beverages
liquor (appetizer)
misc. beverages
2
Instant food products
canned fish
chicken
dental care
dish cleaner
domestic eggs
frozen fish
ham
jam
liver loaf
margarine
meat
meat spreads
seasonal products
soap
sugar
tidbits
turkey
3
detergent
dishes
female sanitary products
4
nut snack
pasta
semi-finished bread
specialty fat
sweet spreads
5
baking powder
berries
canned vegetables
coffee
flour
frozen vegetables
grapes
herbs
mayonnaise
mustard
oil
onions
organic products
packaged fruit/vegetables
pickled vegetables
roll products
rubbing alcohol
salt
soups
vinegar
6
UHT-milk
butter milk
cream cheese
dessert
hard cheese
ice cream
processed cheese
sliced cheese
soft cheese
whipped/sour cream


In [99]:
from collections import Counter

res = Counter(shelves_for_items.values())
for shelf_number in range(len(shelves)):
    print('In shelf #', shelf_number, ', we should add', res[shelf_number], ' items', 'Total:', res[shelf_number]+15) 


In shelf # 0 , we should add 5  items Total: 20
In shelf # 1 , we should add 3  items Total: 18
In shelf # 2 , we should add 18  items Total: 33
In shelf # 3 , we should add 3  items Total: 18
In shelf # 4 , we should add 5  items Total: 20
In shelf # 5 , we should add 20  items Total: 35
In shelf # 6 , we should add 10  items Total: 25
